# Invitae_Bioinformatics_Exercise_B.3
<hr>
By: Barry Guglielmo, ALM
<br>
Date: 10/8/2021
<hr>

### Objective
This code was written in pursute of the Bioinfomatics Engineer, Pipeline position at Invitae.<br>
The objective is to write software that translates transcript coordinates to genomic coordinates.
<br><br>

### Links
<ul>
<li><a href = "Invitae_Bioinformatics_Exercise_B.3.pdf">Exercise PDF</a></li>
<li><a href = "https://www.datahound.org">Personal Portfolio (Python Flask Hosted on Azure)</a></li>
<li><a href = "">GitHub</a></li>
<li><a href = "">LinkedIn</a></li>


In [134]:
import re
import pandas as pd

class cigar():
    '''translates transcript coordinates to genomic coordinates.
    
        Keyword arguments:
        t1 -- path to input cigar strings
        t2 -- list of queries to use
        outpath -- output path for file to be written, must end in / or be blank for local directory
        outname -- name of the output file
        
        Warning:
        Queries longer than the cigar string will be treated as matches
        
    '''
    
    def __init__(self, t1, t2, write = True, outpath = "", outname = "transcript_genome_coordinates.tsv"):
        self.t1 = pd.read_csv(t1, sep = "\t", header = None) # input sequences
        self.t2 = pd.read_csv(t2, sep = "\t", header = None) # queries
        self.tg = [] # array of outputs to be transformed to pandas dataframe at end
        for index, row in self.t2.iterrows():# go through t2 (queries)
            loc = self.t1.loc[self.t1[0]==row[0]].iloc[0] # assuming that each is only used once
            p = re.findall(r'(\d+)([A-Z]{1})', loc[3]) # parse cigar string 
            c = row[1] # transcript coordinate as object to be manipulated without damaging reference
            if c<0: # error handler for negative entries
                print("Warning: Alignment should stat at 0 or positive integer")
            skip = False # a handler if the start point is 0
            for i in range(0,len(p)): # find index of the end of the transcript
                if i == 0 and c == 0: # if the start point is 0 go no further
                    e = i
                    skip = True
                    break
                elif c>=0: # else count down matches and insertions seen (deletions ignored and index will increase)
                    if p[i][1]=='M':
                        c-=int(p[i][0])
                    if p[i][1]=='I':
                        c-=(int(p[i][0]))
                        if c<0:
                            c = -1 # insertions not cumulative
                    if c<=0:
                        e = i
                else:
                    break 
            f = int(loc[2]) # final genome location initialized as the location of start of alignment
            if skip!=True:  # if start point not 0 add to genome location, otherwise skip
                for i in range(0,e+1):  # use the index to calculate the genome location
                    if p[i][1]=='M' or p[i][1]=='D': # matches and deletions counted (insertions ignored but index will increase)
                        f+=int(p[i][0])
                f+=c # add the remainder from calcualting transcript end index (-) to genome location
            self.tg.append([row[0],row[1],loc[1],f])# make list that is now accessible by the class
        self.tg = pd.DataFrame(self.tg) # transform to pandas data frame for ease of use
        if write == True: # write to outfile
            try:
                self.tg.to_csv("%s%s"%(outpath, outname),sep = '\t', index = False, header = False )
            except:
                print("Outpath must end in / or be blank for current directory")
    
    def compare(self, standard):
        '''Do a comparison of self.tg with a known correct output file
        
            Keyword arguments:
            standard -- path to a file with known correct outputs in same format as output from cigar
        '''
        self.standard = pd.read_csv(standard, sep = "\t", header = None)
        outcome = []
        for index, row in self.standard.iterrows():
            d = self.tg.loc[(self.tg[0]==row[0])&(self.tg[1]==row[1])]
            try:
                if list(d.iloc[0]) == list(row):
                    outcome.append("correct,%s,%s"%(list(row),list(d.iloc[0])))
                else:
                    outcome.append("wrong,%s,%s"%(list(row),list(d.iloc[0])))
            except:
                print("Out of Bounds,%s,%s"%(list(row),"No Matching Transcript"))
        return outcome

In [140]:
for i in range(1,6):
    print(i)
    c = cigar("files/t1.tsv","files/tr%i.tsv"%i)
    s = c.compare("files/tr%icorrect.tsv"%i)
    [print(j) for j in s]

1
correct,['TR1', 4, 'CHR1', 7],['TR1', 4, 'CHR1', 7]
correct,['TR1', 7, 'CHR1', 10],['TR1', 7, 'CHR1', 10]
correct,['TR1', 8, 'CHR1', 18],['TR1', 8, 'CHR1', 18]
correct,['TR1', 9, 'CHR1', 19],['TR1', 9, 'CHR1', 19]
correct,['TR1', 12, 'CHR1', 22],['TR1', 12, 'CHR1', 22]
correct,['TR1', 13, 'CHR1', 23],['TR1', 13, 'CHR1', 23]
correct,['TR1', 14, 'CHR1', 23],['TR1', 14, 'CHR1', 23]
correct,['TR1', 15, 'CHR1', 23],['TR1', 15, 'CHR1', 23]
correct,['TR1', 16, 'CHR1', 24],['TR1', 16, 'CHR1', 24]
2
correct,['TR2', 0, 'CHR2', 10],['TR2', 0, 'CHR2', 10]
correct,['TR2', 7, 'CHR2', 17],['TR2', 7, 'CHR2', 17]
3
correct,['TR3', 2, 'CHR3', 5],['TR3', 2, 'CHR3', 5]
correct,['TR3', 3, 'CHR3', 5],['TR3', 3, 'CHR3', 5]
correct,['TR3', 4, 'CHR3', 5],['TR3', 4, 'CHR3', 5]
correct,['TR3', 5, 'CHR3', 6],['TR3', 5, 'CHR3', 6]
correct,['TR3', 6, 'CHR3', 7],['TR3', 6, 'CHR3', 7]
4
correct,['TR4', 2, 'CHR4', 5],['TR4', 2, 'CHR4', 5]
correct,['TR4', 3, 'CHR4', 5],['TR4', 3, 'CHR4', 5]
correct,['TR4', 4, 'CHR4',